In [1]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
import subprocess
from sklearn.utils import shuffle
import itertools

In [2]:
def read_fasta_file(path):
    '''
    used for load fasta data and transformd into numpy.array format
    '''
    fh=open(m6a_benchmark_dataset)
    seq=[]
    for line in fh:
        if line.startswith('>'):
            continue
        else:
            seq.append(line.replace('\r\n',''))
    fh.close()
    matrix_data=np.array([list(e) for e in seq])
    return matrix_data

In [45]:
def AthMethPre_extract_one_line(data_line):
    '''
    extract features from one line, such as one m6A sample
    '''
    A=[0,0,0,1]
    U=[0,0,1,0]
    C=[0,1,0,0]
    G=[1,0,0,0]
    N=[0,0,0,0]
    feature_representation={"A":A,"U":U,"C":C,"G":G,"N":N}
    beginning=0
    end=len(data_line)-1
    alphabet='AUCG'
    feature=[]
    vector=["".join(e) for e in itertools.permutations(alphabet,2)]
    vector.extend(['AA','UU','CC','GG'])
    numerate=[(e[0]*10+e[1])/10.0 for e in itertools.permutations([1,2,3,4],2)]
    numerate.extend([1.1,2.2,3.3,4.4])
    for index,data in enumerate(data_line):
        if "".join((data,data_line[len(data_line)-1])) in vector and index<=23:
            feature.append(numerate[vector.index("".join((data,data_line[len(data_line)-1])))])
    return feature


In [37]:
for e in itertools.combinations_with_replacement("ABCD",2):
    print e

('A', 'A')
('A', 'B')
('A', 'C')
('A', 'D')
('B', 'B')
('B', 'C')
('B', 'D')
('C', 'C')
('C', 'D')
('D', 'D')


In [19]:
def AthMethPre_feature_extraction(matrix_data):
    final_feature_matrix=[AthMethPre_extract_one_line(e) for e in matrix_data]
    return final_feature_matrix

In [47]:

m6a_benchmark_dataset='/home02/chenhuangrong/m6a_data.txt'
matrix_data=read_fasta_file(m6a_benchmark_dataset)
final_feature_matrix=AthMethPre_feature_extraction(matrix_data)
pd.DataFrame(final_feature_matrix).to_csv('/home02/chenhuangrong/Loop_feature.csv',header=None,index=False)




In [12]:
"""
    cross validation
"""
datapath ='/home02/chenhuangrong/PseDNC.csv'
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
Y.extend(Y2)
Y = np.array(Y)
svc = svm.SVC()
# parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,14)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,14))}
parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-5,15,30)), 'gamma':map(lambda x:2**x,np.linspace(-15,5,30))}
clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
C=clf.best_params_['C']
# joblib.dump(clf,'/home02/chenhuangrong/%d_gap_%d_gram.csv'%(gap,gram))
print clf.best_score_
gamma=clf.best_params_['gamma']
y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,),X,Y,cv=10,n_jobs=-1)
print type(y_predict)
y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=-1,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv('/home02/chenhuangrong/PseDNC_predict.csv',header=None,index=False)
ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
#     easy_excel.save("svm_crossvalidation",[str(X.shape[1])],savedata,'/home02/chenhuangrong/cross_validation_frequency_%d_gram.xls'%(gram))
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

0.656465187452
<type 'numpy.ndarray'>
[[['svmC:12596.1013925gamma:0.103244578499', 0.65646518745218052, 0.6423103688239388, 0.7061973986228003, 0.7061973986228003, 0.6067329762815609, 0.6545786809152185, 0.67274052478134105, 0.67274052478134105, 0.31448989282896422, '0.6563', 923.0, 384.0, 514.0, 793.0, 1307.0, 1307.0]]]


In [3]:
def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError

In [4]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [5]:
final_out_to_excel=[]
row0 = [u'特征集', u'样本个数', u'分类器', u'Accuracy', u'Precision', u'Recall', u'SN', u'SP',
                u'Gm', u'F_measure', u'F_score', u'MCC', u'ROC曲线面积', u'tp', u'fn', u'fp', u'tn']
final_out_to_excel.append(row0) #above was used to generate xlsx format Excel file
RNA_code='ACGU' 
interval=3 # RNA_code and interval used mix used to generate AAA AAC AAG ...
division_num=10
divided_num=np.float(division_num)# ten fold so the result should divided by ten


# seq=[e.replace('U','A') for e in seq]
# seq=[e.replace('G','C') for e in seq]
seq=pd.read_csv('/home02/chenhuangrong/PseDNC.csv',header=None,index_col=False)
seq=seq.values

positive_seq=seq[:len(seq)/2]
negative_seq=seq[len(seq)/2:]



#define variable to save data which will be saved later --- begining
y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0
#define variable to save data which will be saved later --- end


#shuffle the data of positive and negative begining
kf = KFold(n_splits=division_num,shuffle=True)  # positive and negative samples will be shuffled 
for train_index , test_index in kf.split(positive_seq):  
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)
#shuffle the data of positive and negative end


    #generate train and test data begining
    X_train = np.array(positive_negative_x_train)
    Y_train = list(map(lambda x: 1, xrange(len(X_train) / 2)))
    Y2_train = list(map(lambda x: 0, xrange(len(X_train) / 2)))
    Y_train.extend(Y2_train)
    Y_train = np.array(Y_train)
    
    X_test = np.array(positive_negative_y_train)
    Y_test  = list(map(lambda x: 1, xrange(len(X_test) / 2)))
    Y2_test  = list(map(lambda x: 0, xrange(len(X_test) / 2)))
    Y_test.extend(Y2_test )
    Y_test = np.array(Y_test)
    #generate train and test data end

    
    # training model and optimized parameters of C and gamma begining
    svc = svm.SVC(probability=True)
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X_train, Y_train)
    # training model and optimized parameters of C and gamma end
    
    
    #print best C and gamma begining
    C=clf.best_params_['C']
    gamma=clf.best_params_['gamma']
    print 'c:',C,'gamma:',gamma
    #print best C and gamma end
    
    
    #getting predict probability and predict label begining
    y_pred_prob=clf.predict_proba(X_test)
    y_pred=clf.predict(X_test)
    #getting predict probability and predict label begining

    #the process of generating  usefule data begining
    y_pred_prob_all.extend(y_pred_prob)
    y_pred_all.extend(y_pred)
    Y_all.extend(Y_test)
    ACC=metrics.accuracy_score(Y_test,y_pred)
    print ACC
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y_test, y_pred) 
    F1_Score=metrics.f1_score(Y_test, y_pred)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y_test, y_pred)

    pos=TP+FN
    neg=FP+TN
    ACC_all=ACC_all+ACC
    print "ACC_all:",ACC_all
    precision_all=precision_all+precision
    recall_all=recall_all+recall
    SN_all=SN_all+SN
    SP_all=SP_all+SP
    GM_all=GM_all+GM
    TP_all=TP_all+TP
    TN_all=TN_all+TN
    FP_all=FP_all+FP
    FN_all=FN_all+FN
    F_measure_all=F_measure_all+F_measure
    F1_Score_all=F1_Score_all+F1_Score
    pos_all=pos_all+pos
    neg_all=neg_all+neg
    MCC_all=MCC_all+MCC
    #the process of generating  usefule data end
    
#the process of save  data to disk begining   
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv('/home02/chenhuangrong/PseDNC_predict.csv',header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T, list(np.array(y_pred_prob_all).astype(list)[:,1]))
roc_auc = auc(fpr, tpr)
savedata=[str(X_train.shape[1]),"正："+str(pos_all)+'负：'+str(neg_all),'svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC_all/divided_num,precision_all/divided_num, recall_all/divided_num,SN_all/divided_num,
            SP_all/divided_num, GM_all/divided_num,F_measure_all/divided_num,F1_Score_all/divided_num,MCC_all/divided_num,roc_auc,TP_all,
            FN_all,FP_all,TN_all]
final_out_to_excel.append(savedata)

print savedata

pd.DataFrame(final_out_to_excel).to_excel("/home02/chenhuangrong/PseDNC.xlsx",sheet_name="independent_test",index=False,header=False)
#the process of save  data to disk end

c: 32.0 gamma: 4.0
0.618320610687
ACC_all: 0.618320610687
c: 32.0 gamma: 4.0
0.641221374046
ACC_all: 1.25954198473
c: 32.0 gamma: 4.0
0.660305343511
ACC_all: 1.91984732824
c: 32.0 gamma: 4.0
0.709923664122
ACC_all: 2.62977099237
c: 32.0 gamma: 4.0
0.614503816794
ACC_all: 3.24427480916
c: 32.0 gamma: 4.0
0.652671755725
ACC_all: 3.89694656489
c: 32.0 gamma: 4.0
0.637404580153
ACC_all: 4.53435114504
c: 32.0 gamma: 4.0
0.665384615385
ACC_all: 5.19973576042
c: 32.0 gamma: 4.0
0.623076923077
ACC_all: 5.8228126835
c: 32.0 gamma: 4.0
0.638461538462
ACC_all: 6.46127422196


NameError: name 'select_num' is not defined

In [6]:
pd.DataFrame(np.matrix(all_y).T).to_csv('/home02/chenhuangrong/PseEIIP_predict.csv',header=None,index=False)